<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/DIRE_preprocess_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ZhendongWang6/DIRE.git

Cloning into 'DIRE'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 63 (delta 7), reused 59 (delta 6), pack-reused 0
Receiving objects: 100% (63/63), 1.95 MiB | 6.52 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt

--2024-03-25 01:03:24--  https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.209.18.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.209.18.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2211383297 (2.1G) [application/octet-stream]
Saving to: ‘256x256_diffusion_uncond.pt’

256x256_diffusion_u 100%[===================>]   2.06G  11.0MB/s    in 3m 19s  

2024-03-25 01:06:44 (10.6 MB/s) - ‘256x256_diffusion_uncond.pt’ saved [2211383297/2211383297]



In [7]:
!pip install ftfy -q
!pip install natsort -q
!pip install tensorboardX -q
!pip install blobfile -q
!pip install mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746507 sha256=435ba8644e3863d78c6f10a3dc54e6f9c6f9247756be479a0dcf3d9041bb8556
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [3]:
!cp /content/drive/MyDrive/DATASETS/real_gen_dataset.zip -d /content

In [4]:
!unzip -q /content/real_gen_dataset.zip -d dataset

In [5]:
%cd DIRE

/content/DIRE


#Process for test set
(We need to remove all lines related to multi-GPU configuration in compe_dire.py before run)

In [ ]:
!python guided-diffusion/compute_dire.py \
--model_path models/256x256_diffusion_uncond.pt \
--attention_resolutions 32,16,8 \
--class_cond False \
--diffusion_steps 1000 --dropout 0.1 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 256 \
--num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True \
--images_dir /content/DIRE/real_gen_dataset/test \
--recons_dir /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/recons_test \
--dire_dir /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/dire_test \
--batch_size 16 --num_samples 1000  --timestep_respacing ddim20 --use_ddim True \
--has_subfolder True

#Process for train set

In [ ]:
import os
image_list = []
extensions = ['.jpg', '.png', '.gif', '.bmp', '.webp']  # Add more extensions as needed
def getname(base_dir):
    for dirpath, dirnames, filenames in os.walk(base_dir):
        if len(filenames) > 0:
            imgs = [filename for filename in filenames if any(ext in filename for ext in extensions)]
            image_list.extend(imgs)
    return image_list


In [ ]:

#Remove existed items before run
data_dir = ''
for dirpath, dirnames, filenames in os.walk(data_dir):
    if len(filenames) >0:
        for file in filenams:
            if file in image_list:
                full_name = os.path.join(dirpath, file)
                os.remove(full_name)
                print('removed :',full_name)


In [ ]:
!python guided-diffusion/compute_dire.py \
--model_path models/256x256_diffusion_uncond.pt \
--attention_resolutions 32,16,8 \
--class_cond False \
--diffusion_steps 1000 --dropout 0.1 --image_size 256 --learn_sigma True --noise_schedule linear --num_channels 256 \
--num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm True \
--images_dir /content/DIRE/real_gen_dataset/train \
--recons_dir /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/recons_train \
--dire_dir /content/drive/MyDrive/DATASETS/DIRE/DiffusionForensics/dire_train \
--batch_size 16 --num_samples 1000  --timestep_respacing ddim20 --use_ddim True \
--has_subfolder True